In [3]:
from dotenv import load_dotenv
from datasets import load_dataset, Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)
load_dotenv()

True

## Prepare Dataset

In [4]:
amnesty_qa_dataset = load_dataset("explodinggradients/amnesty_qa","english_v3")

Repo card metadata block was not found. Setting CardData to empty.


In [8]:
amnesty_qa_dataset

DatasetDict({
    eval: Dataset({
        features: ['user_input', 'reference', 'response', 'retrieved_contexts'],
        num_rows: 20
    })
})

In [9]:
amnesty_qa_dataset['eval'][0]

{'user_input': 'What are the global implications of the USA Supreme Court ruling on abortion?',
 'reference': "The global implications of the USA Supreme Court ruling on abortion are significant. The ruling has led to limited or no access to abortion for one in three women and girls of reproductive age in states where abortion access is restricted. These states also have weaker maternal health support, higher maternal death rates, and higher child poverty rates. Additionally, the ruling has had an impact beyond national borders due to the USA's geopolitical and cultural influence globally. Organizations and activists worldwide are concerned that the ruling may inspire anti-abortion legislative and policy attacks in other countries. The ruling has also hindered progressive law reform and the implementation of abortion guidelines in certain African countries. Furthermore, the ruling has created a chilling effect in international policy spaces, empowering anti-abortion actors to undermine

In [5]:
start_idx = 0
end_idx = 3

questions = []
ground_truths = [] # reference answers
answers = [] # generated answers
contexts = [] # Retrieval contexts


for i in range(start_idx, end_idx):
    questions.append(amnesty_qa_dataset['eval'][i]['user_input'])
    ground_truths.append(amnesty_qa_dataset['eval'][i]['reference'])
    answers.append(amnesty_qa_dataset['eval'][i]['response'])
    contexts.append(amnesty_qa_dataset['eval'][i]['retrieved_contexts'])


# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)


In [18]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 3
})

In [19]:
dataset[0]

{'question': 'What are the global implications of the USA Supreme Court ruling on abortion?',
 'answer': "The global implications of the USA Supreme Court ruling on abortion can be significant, as it sets a precedent for other countries and influences the global discourse on reproductive rights. Here are some potential implications:\n\n1. Influence on other countries: The Supreme Court's ruling can serve as a reference point for other countries grappling with their own abortion laws. It can provide legal arguments and reasoning that advocates for reproductive rights can use to challenge restrictive abortion laws in their respective jurisdictions.\n\n2. Strengthening of global reproductive rights movements: A favorable ruling by the Supreme Court can energize and empower reproductive rights movements worldwide. It can serve as a rallying point for activists and organizations advocating for women's rights, leading to increased mobilization and advocacy efforts globally.\n\n3. Counteracti

In [22]:
result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

Evaluating: 100%|██████████| 12/12 [00:28<00:00,  2.37s/it]


In [24]:
# output to csv
df.to_csv("amnesty_qa_example.csv", index=False)

In [25]:
df

,question,answer,contexts,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,What are the global implications of the USA Su...,The global implications of the USA Supreme Cou...,"[- In 2022, the USA Supreme Court handed down ...",The global implications of the USA Supreme Cou...,1.000000,1.0,1.000000,1.000000
1,Which companies are the main contributors to G...,"According to the Carbon Majors database, the m...","[In recent years, there has been increasing pr...","According to the Carbon Majors database, the m...",1.000000,1.0,0.192308,0.965211
2,Which private companies in the Americas are th...,"According to the Carbon Majors database, the l...",[The issue of greenhouse gas emissions has bec...,The largest private companies in the Americas ...,0.833333,1.0,0.000000,0.989569


In [6]:
sele_idx = 1

txt_out = f"q_{sele_idx}.txt"

with open("q_1.txt", "w") as f:
    f.write("Question:\n" + dataset[sele_idx]['question'] + "\n")
    f.write("Answer:\n" + dataset[sele_idx]['answer'] + "\n")
    for i, context in enumerate(dataset[sele_idx]['contexts']):
        f.write(f"Context {i}:\n" + context + "\n")
    f.write("Ground Truth:\n" + dataset[sele_idx]['ground_truth'] + "\n")